In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from boardgame2 import ReversiEnv
from dynamic_programming import generate_uniform_stochastic_policy, policy_evaluation, stochastic_policy_eval_step, generate_deterministic_policy, deterministic_policy_eval_step
from tree_search import bfs_cannonical
import numpy as np

# Programación dinámica

En esta parte no es necesario la implementación de código ya que ya esta todo resuelto. Si tiene que responder algunas preguntas en **EDX**.

Si lo desea puede ver el código para analizar la implementación, pero es opcional

Si quiere profundizar le recomendamos mirar:

- bfs_cannonical cannonical de la librería tree_search
- policy_evaluation, policy_improve, policy_iterartion y value_iteration de dynamic_programming

### La idea de esta sección es generar las $V^*(s)$y $\Pi^*(s)$ (óptimas) en 4x4 para poder hacer los análisis posteriores
### Por eso se deben correr todas las celdas

# Busqueda de todos los estados canónicos

Solo desde el punto de vista del jugador +1

In [4]:
%%time
board_size = 4
states = bfs_cannonical(board_size, verbose=1)

CPU times: user 2min 4s, sys: 15.8 s, total: 2min 20s
Wall time: 2min


Al ser canónico, no es necesario que el jugador sea parte del estado ya que siempre se puede pensar como que le toca jugar al jugador +1

In [5]:
# Listamos los primeros 5 estados encontrados
for s in list(states.keys())[0:5]:
    print(s)

(0, 0, 0, 0, 0, 1, -1, 0, 0, -1, 1, 0, 0, 0, 0, 0)
(0, 0, -1, 0, 0, -1, -1, 0, 0, 1, -1, 0, 0, 0, 0, 0)
(0, 0, 0, 0, 0, -1, -1, -1, 0, 1, -1, 0, 0, 0, 0, 0)
(0, 0, 0, 0, 0, -1, 1, 0, -1, -1, -1, 0, 0, 0, 0, 0)
(0, 0, 0, 0, 0, -1, 1, 0, 0, -1, -1, 0, 0, -1, 0, 0)


In [6]:
# Guardamos el estado s0
s0 = list(states.keys())[0]

In [7]:
s0

(0, 0, 0, 0, 0, 1, -1, 0, 0, -1, 1, 0, 0, 0, 0, 0)

In [8]:
# Mostrado como tablero
np.array(s0).reshape(4,4)

array([[ 0,  0,  0,  0],
       [ 0,  1, -1,  0],
       [ 0, -1,  1,  0],
       [ 0,  0,  0,  0]], dtype=int8)

Cada estado se guarda con todas sus posibles acciones y dado el estado y la acción, se guarda:
- **next_node**: el próximo estado al ejecutar esa acción
- **done**: si termina el juego (episodio)
- **winner**: si al ejecutar la acción alguno de los dos jugadores gana: (+1 o -1)

In [9]:
for action, next_data in states[s0].items():
    print(f'acción: {action}')
    print(next_data)

acción: (0, 2)
{'done': False, 'winner': -0.0, 'next_node': (0, 0, -1, 0, 0, -1, -1, 0, 0, 1, -1, 0, 0, 0, 0, 0)}
acción: (1, 3)
{'done': False, 'winner': -0.0, 'next_node': (0, 0, 0, 0, 0, -1, -1, -1, 0, 1, -1, 0, 0, 0, 0, 0)}
acción: (2, 0)
{'done': False, 'winner': -0.0, 'next_node': (0, 0, 0, 0, 0, -1, 1, 0, -1, -1, -1, 0, 0, 0, 0, 0)}
acción: (3, 1)
{'done': False, 'winner': -0.0, 'next_node': (0, 0, 0, 0, 0, -1, 1, 0, 0, -1, -1, 0, 0, -1, 0, 0)}


# Ejemplo de un estado terminal

In [10]:
done = 0
for s in states.keys():
    for action, next_data in states[s].items():
        if next_data['done']:
            print(np.array(s).reshape(4,4))
            print(f'acción: {action}')
            print(next_data)
            done = done + 1
            print()
            break
    if done > 5:
        break

[[ 0 -1 -1 -1]
 [ 0  1  1  0]
 [ 1  1  1  0]
 [ 0  0  0  0]]
acción: (-1, 0)
{'done': True, 'winner': 1, 'next_node': (0, 1, 1, 1, 0, -1, -1, 0, -1, -1, -1, 0, 0, 0, 0, 0)}

[[ 0  0  0 -1]
 [ 0  1  1 -1]
 [ 0  1  1 -1]
 [ 0  1  0  0]]
acción: (-1, 0)
{'done': True, 'winner': 1, 'next_node': (0, 0, 0, 1, 0, -1, -1, 1, 0, -1, -1, 1, 0, -1, 0, 0)}

[[ 0  0  1  0]
 [-1  1  1  0]
 [-1  1  1  0]
 [-1  0  0  0]]
acción: (-1, 0)
{'done': True, 'winner': 1, 'next_node': (0, 0, -1, 0, 1, -1, -1, 0, 1, -1, -1, 0, 1, 0, 0, 0)}

[[ 0  0  0  0]
 [ 0  1  1  1]
 [ 0  1  1  0]
 [-1 -1 -1  0]]
acción: (-1, 0)
{'done': True, 'winner': 1, 'next_node': (0, 0, 0, 0, 0, -1, -1, -1, 0, -1, -1, 0, 1, 1, 1, 0)}

[[-1 -1 -1  1]
 [ 0 -1 -1  0]
 [ 0  1 -1 -1]
 [ 0  0  0  0]]
acción: (-1, 0)
{'done': True, 'winner': 1, 'next_node': (1, 1, 1, -1, 0, 1, 1, 0, 0, -1, 1, 1, 0, 0, 0, 0)}

[[-1 -1 -1  0]
 [-1 -1  1  0]
 [ 0  1  1  1]
 [ 0  0  0  0]]
acción: (-1, 0)
{'done': True, 'winner': 1, 'next_node': (1, 1, 1, 0, 1,

La acción (-1, 0) es la acción PASS. En principio solo se ejecuta si no hay opciones válidas

In [11]:
states[(-1, -1, -1, 0, -1, -1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0)]

{(-1, 0): {'done': True,
  'winner': 1,
  'next_node': (1, 1, 1, 0, 1, 1, -1, 0, 0, -1, -1, -1, 0, 0, 0, 0)}}

# Policy Evaluation

### Politica estocástica

In [12]:
stochastic_pi = generate_uniform_stochastic_policy(states)

In [13]:
# ejemplos
stochastic_pi[(0, 0, 0, 0, 0, 1, -1, 0, 0, -1, 1, 0, 0, 0, 0, 0)]

{(0, 2): 0.25, (1, 3): 0.25, (2, 0): 0.25, (3, 1): 0.25}

In [14]:
stochastic_pi[(0, 0, -1, 0, 0, -1, -1, 0, 0, 1, -1, 0, 0, 0, 0, 0)]

{(0, 1): 0.3333333333333333,
 (0, 3): 0.3333333333333333,
 (2, 3): 0.3333333333333333}

Esto genera una política con distribución uniforme que luego será evaluada usuando **policy evaluation**

In [15]:
V_stochastic, iters = policy_evaluation(stochastic_policy_eval_step, 
                             states, 
                             stochastic_pi, 1e-8, verbose=1)

Iteration number:  1 2 3 4 5 6 7 8 9 10 11 12 13 


#### Ejemplos de la V luego de converger

In [16]:
V_stochastic[(0, 0, 0, 0, 0, 1, -1, 0, 0, -1, 1, 0, 0, 0, 0, 0)]

-0.2403001935859148

In [17]:
V_stochastic[(0, 0, -1, 0, 0, -1, -1, 0, 0, 1, -1, 0, 0, 0, 0, 0)]

0.2403001935859148

### Política determinística

In [18]:
det_pi = generate_deterministic_policy(states)

In [19]:
det_pi[(0, 0, 0, 0, 0, 1, -1, 0, 0, -1, 1, 0, 0, 0, 0, 0)]

(3, 1)

In [20]:
det_pi[(0, 0, -1, 0, 0, -1, -1, 0, 0, 1, -1, 0, 0, 0, 0, 0)]

(0, 3)

Notar que ahora la política dado el estado tiene solo una acción posible que se construyó de manera arbitraria

In [21]:
# Run it multiple times to check it takes different number of iterations to converge
V_det, _ = policy_evaluation(deterministic_policy_eval_step, 
                             states, 
                             det_pi, 1e-8, verbose=1)

Iteration number:  1 2 3 4 5 6 7 8 9 10 11 12 


#### Ejemplos de la V luego de converger

In [22]:
V_det[(0, 0, 0, 0, 0, 1, -1, 0, 0, -1, 1, 0, 0, 0, 0, 0)]

1

In [23]:
V_det[(0, 0, -1, 0, 0, -1, -1, 0, 0, 1, -1, 0, 0, 0, 0, 0)]

1

# Policy Iteration

Partiendo de cualquier política (estocástica o determinística), por medio de Policy Iteration se puede obtener las óptimas 

In [24]:
from dynamic_programming import policy_improve, policy_iteration, generate_deterministic_policy

In [25]:
%%time
initial_policy = generate_deterministic_policy(states)
optimum_policy, optimum_V = policy_iteration(states, initial_policy, verbose = 1)

Iteration number:  1 2 3 4 5 6 7 8 9 10 11 12 13 
Number of differences of new policy vs old policy: 12656
---------------------------
Iteration number:  1 2 3 4 5 6 7 8 9 10 11 
Number of differences of new policy vs old policy: 2028
---------------------------
Iteration number:  1 2 3 4 5 6 7 8 9 10 11 12 13 
Number of differences of new policy vs old policy: 515
---------------------------
Iteration number:  1 2 3 4 5 6 7 8 9 10 11 12 13 
Number of differences of new policy vs old policy: 110
---------------------------
Iteration number:  1 2 3 4 5 6 7 8 9 10 11 12 13 
Number of differences of new policy vs old policy: 30
---------------------------
Iteration number:  1 2 3 4 5 6 7 8 9 10 11 12 13 
Number of differences of new policy vs old policy: 10
---------------------------
Iteration number:  1 2 3 4 5 6 7 8 9 10 11 12 13 
Number of differences of new policy vs old policy: 3
---------------------------
Iteration number:  1 2 3 4 5 6 7 8 9 10 11 12 13 
Number of differences of n

In [26]:
np.save('mdp/pi_mdp', optimum_policy)
np.save('mdp/v_mdp', optimum_V)

In [27]:
optimum_policy[(0, 0, -1, 0, 0, -1, -1, 0, 0, 1, -1, 0, 0, 0, 0, 0)]

(0, 3)

In [28]:
optimum_V[(0, 0, -1, 0, 0, -1, -1, 0, 0, 1, -1, 0, 0, 0, 0, 0)]

1

In [29]:
np.array((0, 0, -1, 0, 0, -1, -1, 0, 0, 1, -1, 0, 0, 0, 0, 0)).reshape(4, 4)

array([[ 0,  0, -1,  0],
       [ 0, -1, -1,  0],
       [ 0,  1, -1,  0],
       [ 0,  0,  0,  0]])

# Value Iteration

In [30]:
from dynamic_programming import value_iteration

In [31]:
%%time
V, delta = value_iteration(states, verbose=1)

1 16 2.148329015302604
2 14 1.3984082309742596
3 14 0.7103688654451921
4 13 0.3661814318465639
5 12 0.1380402974781458
6 11 0.05770628692848223
7 10 0.02005554416506682
8 8 0.006710033363777003
9 6 0.0023857896404540454
10 6 0.0005964474101135114
11 6 0.00011183388939628339
12 0 0.0
CPU times: user 7.8 s, sys: 0 ns, total: 7.8 s
Wall time: 7.79 s


# Cuestionario:

### Pregunta 1:

Dado el siguiente estado:

(1, 1, 1, -1, 0, 1, 1, 0, 0, -1, 1, 1, 0, 0, 0, 0)

y luego de ejecutar la acción:

(3, 0)

Evaluar **states** e indicar:

¿Cuánto vale **done**?

In [54]:
state = (1, 1, 1, -1, 0, 1, 1, 0, 0, -1, 1, 1, 0, 0, 0, 0)
print(np.array(state).reshape(4,4))

[[ 1  1  1 -1]
 [ 0  1  1  0]
 [ 0 -1  1  1]
 [ 0  0  0  0]]


In [55]:
states[state]

{(2, 0): {'done': False,
  'winner': -0.0,
  'next_node': (-1, -1, -1, 1, 0, -1, -1, 0, -1, -1, -1, -1, 0, 0, 0, 0)},
 (3, 0): {'done': True,
  'winner': -1,
  'next_node': (-1, -1, -1, 1, 0, -1, -1, 0, 0, -1, -1, -1, -1, 0, 0, 0)},
 (3, 1): {'done': False,
  'winner': -0.0,
  'next_node': (-1, -1, -1, 1, 0, -1, -1, 0, 0, -1, -1, -1, 0, -1, 0, 0)}}

In [56]:
states[state][(3, 0)]

{'done': True,
 'winner': -1,
 'next_node': (-1, -1, -1, 1, 0, -1, -1, 0, 0, -1, -1, -1, -1, 0, 0, 0)}

### Pregunta 2:

Luego de generara una política estocástica con distribución uniforme.

Dado el estado (0, 0, -1, 0, 0, -1, -1, 0, 0, 1, -1, 0, 0, 0, 0, 0)

¿Cuánto es la probabilidad de la acción (0, 3) ?

¿Cuánto es la probabilidad de la acción (3, 3) ?

In [57]:
state = (0, 0, -1, 0, 0, -1, -1, 0, 0, 1, -1, 0, 0, 0, 0, 0)
print(np.array(state).reshape(4,4))

[[ 0  0 -1  0]
 [ 0 -1 -1  0]
 [ 0  1 -1  0]
 [ 0  0  0  0]]


In [58]:
stochastic_pi = generate_uniform_stochastic_policy(states)

In [59]:
stochastic_pi[state]

{(0, 1): 0.3333333333333333,
 (0, 3): 0.3333333333333333,
 (2, 3): 0.3333333333333333}

### Pregunta 3:

Luego de evaluar las dos políticas (correr policy_evaluation) (determinística y estocástica) se pueden sacar las siguientes conclusiones:

La evaluación de cualquier estado de la política determinística siempre da 1, 0, o -1.

La evaluación de cualquier estado de la política estocástica siempre da 1, 0, o -1.

La evaluación de cualquier estado de la política determinística no tiene por que dar un número entero.

La evaluación de cualquier estado de la política estocástica no tiene por que dar un número entero.

In [60]:
V_det, _ = policy_evaluation(deterministic_policy_eval_step, 
                             states, 
                             det_pi, 1e-8, verbose=1)

Iteration number:  1 2 3 4 5 6 7 8 9 10 11 12 


In [62]:
V_stochastic, _ = policy_evaluation(stochastic_policy_eval_step, 
                             states, 
                             stochastic_pi, 1e-8, verbose=1)

Iteration number:  1 2 3 4 5 6 7 8 9 10 11 12 13 


In [69]:
np.unique(list(V_det.values()))

array([-1,  0,  1])

In [71]:
np.unique(list(V_stochastic.values()))[:10]

array([-1.        , -0.98611111, -0.98611111, -0.97916667, -0.97569444,
       -0.97222222, -0.97222222, -0.96875   , -0.95833333, -0.94814815])

### Pregunta 4:

Luego de correr policy iteration. Evaluar el estado:

(0, 0, -1, 0, 0, -1, -1, 0, 0, 1, -1, 0, 0, 0, 0, 0)

¿Cuánto vale la value de ese estado?

¿Cual es la jugada ganadora en ese estado?

In [72]:
state = (0, 0, -1, 0, 0, -1, -1, 0, 0, 1, -1, 0, 0, 0, 0, 0)
print(np.array(state).reshape(4,4))

[[ 0  0 -1  0]
 [ 0 -1 -1  0]
 [ 0  1 -1  0]
 [ 0  0  0  0]]


In [73]:
optimum_V[state]

1

In [74]:
optimum_policy[state]

(0, 3)

### Pregunta 5:

Dado un rápido análisis de la V y PI optimas, indicar cual de las siguientes afirmaciones son correctas:

El que juega segundo gana solo si juega en una posición que come en diagonal.

El que juega segundo gana independientemente de lo que juegue.

Si el primero juega bien, puede lograr un empate.

Si el segundo no juega bien en su primer turno, y el primero juega bien en su segundo turno, éste (el primero) ganará la partida. 

In [59]:
def print_state(state, values, policy, player, inv = 1):
    current_player_wins = values[state] == 1
    other_player = 3 - player
    win_player = player if current_player_wins else other_player
    print((inv * np.array(state)).reshape(4,4))
    print(f"Winner will be player: {win_player}")
    print(f"Best move: {policy[state]}")

In [60]:
state = (0, 0, 0, 0, 0, 1, -1, 0, 0, -1, 1, 0, 0, 0, 0, 0)
print_state(state, optimum_V, optimum_policy, 1)

[[ 0  0  0  0]
 [ 0  1 -1  0]
 [ 0 -1  1  0]
 [ 0  0  0  0]]
Winner will be player: 2
Best move: (0, 2)


In [63]:
def print_game_tree(states, values, op_policy, state = (0, 0, 0, 0, 0, 1, -1, 0, 0, -1, 1, 0, 0, 0, 0, 0), player = 1, round = 0, action=None, max_rounds = 3):
    if (round > max_rounds):
        return
    
    inv = 1 + (1-player)*2
    next_player = 3 - player
    
    print("#"*40)
    print("Current Round: ", "Initial" if (round == 0) else round)
    print("Current player: ", player)
    if (action is not None):
        print("Prev move: ", action)
    print("Current board: ")
    print_state(state, values, op_policy, player, inv)
    print("#"*40)
    print()
    
    actions = states[state]
    next_round = round + 1
    
    print(f"Posible moves for player: {player}")
    for (a, data) in actions.items():
        next_state = data['next_node']
        print("If it moves to: ", a)
        print_state(next_state, values, op_policy, next_player, inv * -1)
        print()
    
    for (a, data) in actions.items():
        next_state = data['next_node']
        done = data['done']
        if (not done):
            print_game_tree(states, values, op_policy, state=next_state, player=next_player, round = next_round, action = a, max_rounds = max_rounds)

In [64]:
print_game_tree(states, optimum_V, optimum_policy, max_rounds = 1)

########################################
Current Round:  Initial
Current player:  1
Current board: 
[[ 0  0  0  0]
 [ 0  1 -1  0]
 [ 0 -1  1  0]
 [ 0  0  0  0]]
Winner will be player: 2
Best move: (0, 2)
########################################

Posible moves for player: 1
If it moves to:  (0, 2)
[[ 0  0  1  0]
 [ 0  1  1  0]
 [ 0 -1  1  0]
 [ 0  0  0  0]]
Winner will be player: 2
Best move: (0, 3)

If it moves to:  (1, 3)
[[ 0  0  0  0]
 [ 0  1  1  1]
 [ 0 -1  1  0]
 [ 0  0  0  0]]
Winner will be player: 2
Best move: (0, 3)

If it moves to:  (2, 0)
[[ 0  0  0  0]
 [ 0  1 -1  0]
 [ 1  1  1  0]
 [ 0  0  0  0]]
Winner will be player: 2
Best move: (3, 0)

If it moves to:  (3, 1)
[[ 0  0  0  0]
 [ 0  1 -1  0]
 [ 0  1  1  0]
 [ 0  1  0  0]]
Winner will be player: 2
Best move: (3, 0)

########################################
Current Round:  1
Current player:  2
Prev move:  (0, 2)
Current board: 
[[ 0  0  1  0]
 [ 0  1  1  0]
 [ 0 -1  1  0]
 [ 0  0  0  0]]
Winner will be player: 2
Best move: 